In [2]:
# 从零开始实现整个方法，包括数据流水线，模型，损失函数和小批量随机梯度下降优化器。
%matplotlib inline
import random
import torch
# from d2l import torch as d2l

In [3]:
# 根据带有噪声的线性模型构造一个人造数据集。
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w)+b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))


true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [4]:
# features中的每一行都包含一个二维数据样本
# labels中的每一行都包含一维标签值（一个标量）
print('features:', features[0], '\nlabel:', labels[0])

features: tensor([-1.0536,  0.0718]) 
label: tensor([1.8617])


In [ ]:
d2l.set_figsize()
d2l.plt.scatter(features[:, 1].detach().numpy(), labels.detach().numpy(), 1);

In [5]:
# 该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为batch_size的小批量。
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
        # python的iterator,不断地调用不断地返回，相当于每次返回一个x和y.
        yield features[batch_indices], labels[batch_indices]

batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break
# 给我一些样本标号，每一次我随机地选取一些样本参与计算。

tensor([[ 0.7730,  0.6866],
        [ 0.7863, -0.2135],
        [-0.9264, -1.0832],
        [-0.6853,  2.3659],
        [ 0.1756,  1.7411],
        [ 0.4607,  1.7960],
        [-0.2603, -1.1559],
        [ 0.3918, -1.5025],
        [-1.6277,  0.0622],
        [ 0.7657,  0.0128]]) 
 tensor([[ 3.4031],
        [ 6.4942],
        [ 6.0325],
        [-5.2160],
        [-1.3665],
        [-0.9777],
        [ 7.6049],
        [10.0996],
        [ 0.7425],
        [ 5.6843]])


In [6]:
# 定义初始化模型参数
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [7]:
# 定义线性回归模型
def linreg(X, w, b):
    return torch.matmul(X, w)+b

In [8]:
# 定义损失函数
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [9]:
# 定义优化算法
# 梯度会存在.grad里面
def sgd(params, lr, batch_size):
    # 更新的时候不需要参与梯度计算
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            # pytorch不会自动地帮你把梯度设为0，所以你需要手动地设为0
            # 这样下一次计算梯度的时候，就不会与上一次相关了
            param.grad.zero_()

In [10]:
lr = 0.03
# 整个数据扫三遍
num_epochs = 3 
net = linreg
loss = squared_loss

# 每一次对数据扫一遍
for epoch in range(num_epochs):
    # 每一次拿出批量大小的X,y
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y) # X和y的小批量损失 （batch_size, l）
        l.sum().backward()
        # 使用参数的梯度更新参数
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch+1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.043398
epoch 2, loss 0.000174
epoch 3, loss 0.000052


In [11]:
print(f'w的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - b}')

w的估计误差：tensor([-8.0109e-05, -4.3416e-04], grad_fn=<SubBackward0>)
b的估计误差：tensor([-6.0081e-05], grad_fn=<RsubBackward1>)


In [12]:
import numpy as np
from torch.utils import data
import torch
# from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [19]:
#调用框架中现有的API来读取数据
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个PyTorch数据迭代器
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[-0.7364, -0.0919],
         [-0.1959, -0.6074],
         [ 0.2787,  0.5294],
         [ 0.8559,  0.4535],
         [-0.2042,  1.6201],
         [ 0.4883, -1.3968],
         [ 0.1869, -0.8416],
         [ 0.4939,  0.4296],
         [-0.4608, -1.6862],
         [ 1.0066,  1.0389]]),
 tensor([[ 3.0321],
         [ 5.8816],
         [ 2.9606],
         [ 4.3645],
         [-1.7183],
         [ 9.9206],
         [ 7.4383],
         [ 3.7195],
         [ 9.0081],
         [ 2.6563]])]

In [14]:
# 使用框架的预定义好的层
from torch import nn
# Sequential container, list of layer
net = nn.Sequential(nn.Linear(2, 1))

In [15]:
# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [16]:
# 计算均方误差使用的是MSELoss类
loss = nn.MSELoss()

In [17]:
# 实例化SGD实例
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [21]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        # 先把优化器的梯度清零
        trainer.zero_grad()
        # 计算梯度
        l.backward()
        # 更新模型
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000105
epoch 2, loss 0.000105
epoch 3, loss 0.000106
